In [1]:
import leidenalg
import scanpy, phate
import numpy as np
import scprep
import matplotlib.pyplot as plt
import pandas as pd
import gspa
import magic
from scipy.spatial.distance import cdist

2024-03-21 17:04:01.475522: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-21 17:04:01.477383: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-21 17:04:01.516420: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-21 17:04:01.517272: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-21 17:04:07.885447: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
acute = scanpy.read_h5ad('data/acute_tcells.h5ad')
chronic = scanpy.read_h5ad('data/chronic_tcells.h5ad')
adata = scanpy.concat((acute,chronic))
adata.obs['batch'] = adata.obs['batch'].astype('category')

In [3]:
phate_op = phate.PHATE(random_state=42, n_jobs=-1, knn=30)
adata.obsm['X_phate'] = phate_op.fit_transform(adata.to_df())

Calculating PHATE...
  Running PHATE on 39704 observations and 14152 variables.
  Calculating graph and diffusion operator...
    Calculating PCA...
    Calculated PCA in 39.26 seconds.
    Calculating KNN search...
    Calculated KNN search in 273.00 seconds.
    Calculating affinities...
    Calculated affinities in 36.68 seconds.
  Calculated graph and diffusion operator in 350.87 seconds.
  Calculating landmark operator...
    Calculating SVD...
    Calculated SVD in 38.37 seconds.
    Calculating KMeans...
    Calculated KMeans in 14.77 seconds.
  Calculated landmark operator in 57.19 seconds.
  Calculating optimal t...
    Automatically selected t = 20
  Calculated optimal t in 5.70 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 1.40 seconds.
  Calculating metric MDS...
  Calculated metric MDS in 10.48 seconds.
Calculated PHATE in 425.67 seconds.


In [4]:
data, data_hvgs = scprep.select.highly_variable_genes(adata.to_df(), adata.var_names, percentile=90)
data_hvg = data[data_hvgs]
data_hvg = data_hvg / np.linalg.norm(data_hvg, axis=0)

In [5]:
uniform_signal = np.ones((1, adata.n_obs))
uniform_signal = uniform_signal / np.linalg.norm(uniform_signal, axis=1).reshape(-1,1)

In [6]:
results = {}
magic_op = magic.MAGIC()
magic_op.graph = phate_op.graph.to_pygsp()
cell_dictionary = magic_op.diff_op.todense()

In [9]:
signals_projected = gspa.embedding.project(data_hvg.T, cell_dictionary)
signals_reduced = gspa.embedding.svd(signals_projected)
results['signal_embedding'] = gspa.embedding.run_ae(signals_reduced)
uniform_projected = gspa.embedding.project(uniform_signal, cell_dictionary)
results['localization_score'] = cdist(uniform_projected, signals_projected).reshape(-1,)

2024-03-21 17:17:28.175866: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


Epoch 1/100
43/43 [==============================] - 2s 27ms/step - loss: 0.0015 - val_loss: 0.0010
Epoch 2/100
43/43 [==============================] - 1s 16ms/step - loss: 6.7313e-04 - val_loss: 5.6411e-04
Epoch 3/100
43/43 [==============================] - 0s 7ms/step - loss: 4.5315e-04 - val_loss: 4.5827e-04
Epoch 4/100
43/43 [==============================] - 0s 10ms/step - loss: 3.6756e-04 - val_loss: 3.9821e-04
Epoch 5/100
43/43 [==============================] - 0s 7ms/step - loss: 3.2841e-04 - val_loss: 3.5193e-04
Epoch 6/100
43/43 [==============================] - 0s 6ms/step - loss: 2.9103e-04 - val_loss: 3.3258e-04
Epoch 7/100
43/43 [==============================] - 1s 17ms/step - loss: 2.6508e-04 - val_loss: 3.4386e-04
Epoch 8/100
43/43 [==============================] - 0s 6ms/step - loss: 2.8261e-04 - val_loss: 3.0662e-04
Epoch 9/100
43/43 [==============================] - 0s 6ms/step - loss: 2.4429e-04 - val_loss: 2.8101e-04
Epoch 10/100
43/43 [=====================

### cluster analysis

In [10]:
gene_phate_op = phate.PHATE(random_state=0)
data_phate = gene_phate_op.fit_transform(results['signal_embedding'])

Calculating PHATE...
  Running PHATE on 1416 observations and 128 variables.
  Calculating graph and diffusion operator...
    Calculating PCA...
    Calculated PCA in 0.18 seconds.
    Calculating KNN search...
    Calculated KNN search in 0.42 seconds.
    Calculating affinities...
    Calculated affinities in 0.03 seconds.
  Calculated graph and diffusion operator in 0.65 seconds.
  Calculating optimal t...
    Automatically selected t = 21
  Calculated optimal t in 1.75 seconds.
  Calculating diffusion potential...
  Calculated diffusion potential in 0.60 seconds.
  Calculating metric MDS...
  Calculated metric MDS in 2.56 seconds.
Calculated PHATE in 5.60 seconds.


In [11]:
gene_clusters = np.array(leidenalg.find_partition(gene_phate_op.graph.to_igraph(), 
                                                  leidenalg.ModularityVertexPartition,
                                                  seed=0).membership)
gene_clusters = pd.Series(gene_clusters, index=data_hvgs.values)

In [12]:
np.savez('./results/MAGIC_gene_embedding.npz', 
         signal_embedding=results['signal_embedding'],
         localization_score=results['localization_score'],
         genes=data_hvgs.values, clusters=gene_clusters.values)